In [9]:
import time
from datetime import datetime, timedelta
import os
import threading
import tkinter as tk
from tkinter import messagebox

class AlarmClock:
    def __init__(self, root):
        self.alarms = []
        self.root = root
        self.root.title("Alarm Clock")

        # GUI elements
        self.time_label = tk.Label(root, text="Enter alarm time (HH:MM):")
        self.time_label.pack()
        self.time_entry = tk.Entry(root)
        self.time_entry.pack()
        
        self.tone_label = tk.Label(root, text="Enter tone file name:")
        self.tone_label.pack()
        self.tone_entry = tk.Entry(root)
        self.tone_entry.pack()

        self.snooze_label = tk.Label(root, text="Enter snooze duration (minutes):")
        self.snooze_label.pack()
        self.snooze_entry = tk.Entry(root)
        self.snooze_entry.pack()

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.pack()

        self.run_button = tk.Button(root, text="Start Alarms", command=self.run)
        self.run_button.pack()

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.tone_entry.get()
        snooze_duration = int(self.snooze_entry.get())
        alarm = {
            'time': alarm_time,
            'tone': tone,
            'snooze_duration': snooze_duration,
            'snoozed_until': None
        }
        self.alarms.append(alarm)
        messagebox.showinfo("Alarm Added", f"Alarm set for {alarm_time} with tone '{tone}'")

    def check_alarms(self):
        while True:
            current_time = datetime.now().strftime("%H:%M")
            for alarm in self.alarms:
                alarm_time = alarm['snoozed_until'] if alarm['snoozed_until'] else alarm['time']
                if current_time == alarm_time:
                    messagebox.showinfo("Alarm", "Wake up!")
                    self.play_sound(alarm['tone'])
                    self.snooze_alarm(alarm)
            time.sleep(30)  # Check every 30 seconds to save resources

    def play_sound(self, sound_file):
        if os.name == 'nt':  # Windows
            os.system(f'start {sound_file}')
        elif os.name == 'posix':  # macOS or Linux
            os.system(f'open {sound_file}') if os.uname().sysname == 'Darwin' else os.system(f'aplay {sound_file}')

    def snooze_alarm(self, alarm):
        snooze_until = (datetime.now() + timedelta(minutes=alarm['snooze_duration'])).strftime("%H:%M")
        alarm['snoozed_until'] = snooze_until
        messagebox.showinfo("Snooze", f"Alarm snoozed until {snooze_until}")

    def run(self):
        threading.Thread(target=self.check_alarms).start()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmClock(root)
    root.mainloop()


In [18]:
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime, timedelta
import threading
import time
import pygame

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Initialize pygame mixer
        pygame.mixer.init()
        
        # Alarm list
        self.alarms = []
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM:SS)").grid(row=0, column=0, padx=10, pady=5)
        self.time_entry = tk.Entry(root)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Tone").grid(row=1, column=0, padx=10, pady=5)
        self.tone_path = tk.StringVar()
        self.tone_entry = tk.Entry(root, textvariable=self.tone_path)
        self.tone_entry.grid(row=1, column=1, padx=10, pady=5)
        
        self.browse_button = tk.Button(root, text="Browse", command=self.browse_tone)
        self.browse_button.grid(row=1, column=2, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=3, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def browse_tone(self):
        filepath = filedialog.askopenfilename(filetypes=[("Audio Files", "*.mp3;*.wav")])
        if filepath:
            self.tone_path.set(filepath)

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.tone_path.get()

        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            if tone:
                self.alarms.append((alarm_time, tone))
                self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with tone {tone}")
                messagebox.showinfo("Success", "Alarm added successfully!")
            else:
                messagebox.showwarning("Error", "Please select a tone!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self, tone):
        pygame.mixer.music.load(tone)
        pygame.mixer.music.play()

    def stop_tone(self):
        pygame.mixer.music.stop()

    def snooze_alarm(self):
        selected = self.alarm_listbox.curselection()
        if selected:
            alarm = self.alarms[selected[0]]
            alarm_time = datetime.strptime(alarm[0], "%H:%M:%S") + timedelta(minutes=5)
            snoozed_time = alarm_time.time().strftime("%H:%M:%S")

            self.alarms.append((snoozed_time, alarm[1]))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time} with tone {alarm[1]}")

            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm selected to snooze!")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm in self.alarms:
                if alarm[0] == current_time:
                    self.play_tone(alarm[1])
                    messagebox.showinfo("Alarm", f"Alarm ringing at {alarm[0]}!")
                    self.stop_tone()
                    self.alarms.remove(alarm)
                    self.alarm_listbox.delete(0, tk.END)
                    for a in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {a[0]} with tone {a[1]}")
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()

ModuleNotFoundError: No module named 'pygame'

In [ ]:
import date
import

In [22]:
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime, timedelta
import threading
import time
import winsound

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM:SS)").grid(row=0, column=0, padx=10, pady=5)
        self.time_entry = tk.Entry(root)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Tone").grid(row=1, column=0, padx=10, pady=5)
        self.tone_path = tk.StringVar()
        self.tone_entry = tk.Entry(root, textvariable=self.tone_path)
        self.tone_entry.grid(row=1, column=1, padx=10, pady=5)
        
        self.browse_button = tk.Button(root, text="Browse", command=self.browse_tone)
        self.browse_button.grid(row=1, column=2, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=3, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def browse_tone(self):
        filepath = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if filepath:
            self.tone_path.set(filepath)

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.tone_path.get()

        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            if tone:
                self.alarms.append((alarm_time, tone))
                self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with tone {tone}")
                messagebox.showinfo("Success", "Alarm added successfully!")
            else:
                messagebox.showwarning("Error", "Please select a tone!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self, tone):
        try:
            winsound.PlaySound(tone, winsound.SND_FILENAME | winsound.SND_ASYNC)
        except RuntimeError:
            messagebox.showwarning("Error", "Unable to play the selected tone!")

    def stop_tone(self):
        winsound.PlaySound(None, winsound.SND_PURGE)

    def snooze_alarm(self):
        selected = self.alarm_listbox.curselection()
        if selected:
            alarm = self.alarms[selected[0]]
            alarm_time = datetime.strptime(alarm[0], "%H:%M:%S") + timedelta(minutes=5)
            snoozed_time = alarm_time.time().strftime("%H:%M:%S")

            self.alarms.append((snoozed_time, alarm[1]))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time} with tone {alarm[1]}")

            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm selected to snooze!")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm in self.alarms:
                if alarm[0] == current_time:
                    self.play_tone(alarm[1])
                    messagebox.showinfo("Alarm", f"Alarm ringing at {alarm[0]}!")
                    self.stop_tone()
                    self.alarms.remove(alarm)
                    self.alarm_listbox.delete(0, tk.END)
                    for a in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {a[0]} with tone {a[1]}")
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()

In [22]:


import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime, timedelta
import threading
import time  
!pip install playsound

# Then run your script again as usual.
from playsound import playsound





class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root,width=20)
        root.columnconfigure(1, weight=0)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Tone").grid(row=1, column=0, padx=10, pady=5)
        self.tone_path = tk.StringVar()
        self.tone_entry = tk.Entry(root, textvariable=self.tone_path)
        self.tone_entry.grid(row=1, column=1, padx=10, pady=5)
        
        self.browse_button = tk.Button(root, text="Browse", command=self.browse_tone)
        self.browse_button.grid(row=1, column=2, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=3, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def browse_tone(self):
        filepath = filedialog.askopenfilename(filetypes=[("Audio Files", "*.mp3;*.wav")])
        if filepath:
            self.tone_path.set(filepath)

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.tone_path.get()

        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            if tone:
                self.alarms.append((alarm_time, tone))
                self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with tone {tone}")
                messagebox.showinfo("Success", "Alarm added successfully!")
            else:
                messagebox.showwarning("Error", "Please select a tone!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self, tone):
        try:
            playsound(tone)
        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the selected tone: {e}")

    def snooze_alarm(self):
        selected = self.alarm_listbox.curselection()
        if selected:
            alarm = self.alarms[selected[0]]
            alarm_time = datetime.strptime(alarm[0], "%H:%M:%S") + timedelta(minutes=5)
            snoozed_time = alarm_time.time().strftime("%H:%M:%S")

            self.alarms.append((snoozed_time, alarm[1]))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time} with tone {alarm[1]}")

            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm selected to snooze!")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm in self.alarms:
                if alarm[0] == current_time:
                    self.play_tone(alarm[1])
                    messagebox.showinfo("Alarm", f"Alarm ringing at {alarm[0]}!")
                    self.alarms.remove(alarm)
                    self.alarm_listbox.delete(0, tk.END)
                    for a in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {a[0]} with tone {a[1]}")
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [26]:
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime, timedelta
import threading
import time
import base64
from playsound import playsound
import os

# Embed the audio file as a base64 string (example: alarm.mp3)
embedded_audio = """
<D:\alarm_sound>
"""

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM:SS):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=1, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=2, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=3, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def save_embedded_audio(self):
        """Decode the embedded audio and save it as a temporary file."""
        temp_file = "temp_alarm.mp3"
        with open(temp_file, "wb") as f:
            f.write(base64.b64decode(embedded_audio))
        return temp_file

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            self.alarms.append(alarm_time)
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self):
        try:
            temp_file = self.save_embedded_audio()
            playsound(temp_file)
            os.remove(temp_file)  # Clean up temporary file
        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the alarm sound: {e}")

    def snooze_alarm(self):
        selected = self.alarm_listbox.curselection()
        if selected:
            alarm_time = self.alarms[selected[0]]
            snoozed_time = (datetime.strptime(alarm_time, "%H:%M:%S") + timedelta(minutes=5)).time().strftime("%H:%M:%S")
            self.alarms.append(snoozed_time)
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm selected to snooze!")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            if current_time in self.alarms:
                self.play_tone()
                messagebox.showinfo("Alarm", f"Alarm ringing at {current_time}!")
                self.alarms.remove(current_time)
                self.alarm_listbox.delete(0, tk.END)
                for alarm in self.alarms:
                    self.alarm_listbox.insert(tk.END, f"Alarm at {alarm}")
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [32]:
import tkinter as tk
from tkinter import messagebox, filedialog, ttk
from datetime import datetime, timedelta
import threading
import time
import base64
from playsound import playsound
import os

# Embed the audio file as a base64 string (example: alarm.mp3)
embedded_audio_1 = """
<"D:\alarm_sound\alarm_1.mp3">
"""
embedded_audio_2 = """
<"D:\alarm_sound\alarm_2.mp3">
"""
embedded_audio_3 = """
<"D:\alarm_sound\morning_alarm.mp3">
"""
embedded_audio_4 = """
<"D:\alarm_sound\old_alarm_sound.mp3">
"""

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10, columnspan=2)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=2, padx=10, pady=5, columnspan=2)
        
        tk.Label(root, text="Select Sound:").grid(row=1, column=0, padx=5, pady=10, columnspan=2)
        self.sound_options = ["Sound 1", "Sound 2", "Sound 3","Sound 4"]
        self.selected_sound = tk.StringVar(root)
        self.selected_sound.set(self.sound_options[0])
        self.sound_menu = ttk.Combobox(root, textvariable=self.selected_sound, values=self.sound_options)
        self.sound_menu.grid(row=1, column=2, padx=10, pady=5, columnspan=2)
        
        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=2, pady=10, columnspan=2)
        
        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=4, pady=10)
        
        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=2, pady=10, columnspan=2)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def save_embedded_audio(self, audio_data, D:\alarm_sound):
        """Decode the embedded audio and save it as a temporary file."""
        with open(D:\alarm_sound, "wb") as f:
            f.write(base64.b64decode(audio_data))
        return filename

    def get_audio_file(self):
        """Return the file path for the selected alarm sound."""
        if self.selected_sound.get() == "Sound 1":
            return self.save_embedded_audio(embedded_audio_1, "temp_alarm_1.mp3")
        elif self.selected_sound.get() == "Sound 2":
            return self.save_embedded_audio(embedded_audio_2, "temp_alarm_2.mp3")
        else:
            return self.save_embedded_audio(embedded_audio_3, "temp_alarm_3.mp3")

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M")
            self.alarms.append((alarm_time, self.selected_sound.get()))
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with {self.selected_sound.get()}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM.")

    def play_tone(self):
        try:
            audio_file = self.get_audio_file()
            playsound(audio_file)
            os.remove(audio_file)  # Clean up temporary file
        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the alarm sound: {e}")

    def snooze_alarm(self):
        selected = self.alarm_listbox.curselection()
        if selected:
            alarm_time, sound = self.alarms[selected[0]]
            snoozed_time = (datetime.strptime(alarm_time, "%H:%M:%S") + timedelta(minutes=5)).time().strftime("%H:%M:%S")
            self.alarms.append((snoozed_time, sound))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time} with {sound}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm selected to snooze!")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm_time, sound in self.alarms:
                if current_time == alarm_time:
                    self.play_tone()
                    messagebox.showinfo("Alarm", f"Alarm ringing at {current_time} with {sound}!")
                    self.alarms.remove((alarm_time, sound))
                    self.alarm_listbox.delete(0, tk.END)
                    for at, s in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {at} with {s}")
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


<>:17: SyntaxWarning: invalid escape sequence '\m'
<>:20: SyntaxWarning: invalid escape sequence '\o'
<>:17: SyntaxWarning: invalid escape sequence '\m'
<>:20: SyntaxWarning: invalid escape sequence '\o'
C:\Users\angqe\AppData\Local\Temp\ipykernel_11484\2084843075.py:17: SyntaxWarning: invalid escape sequence '\m'
  embedded_audio_3 = """
C:\Users\angqe\AppData\Local\Temp\ipykernel_11484\2084843075.py:20: SyntaxWarning: invalid escape sequence '\o'
  embedded_audio_4 = """


SyntaxError: unexpected character after line continuation character (2084843075.py, line 59)

In [ ]:
import tkinter as tk
from tkinter import messagebox
from datetime import datetime, timedelta
import threading
import time
import base64
from playsound import playsound
import os

# Embed the audio file as a base64 string
embedded_audio_1 = """
<"D:\alarm_sound\alarm_1.mp3">
"""
embedded_audio_2 = """
<"D:\alarm_sound\alarm_2.mp3">
"""
embedded_audio_3 = """
<"D:\alarm_sound\morning_alarm.mp3">
"""
embedded_audio_4 = """
<"D:\alarm_sound\old_alarm_sound.mp3">
"""

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        self.currently_playing = False  # Track if an alarm is currently sounding
        self.stop_signal = threading.Event()

        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=1, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=2, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=3, column=0, pady=10)

        self.stop_button = tk.Button(root, text="Stop", command=self.stop_alarm)
        self.stop_button.grid(row=3, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def save_embedded_audio(self):
        """Decode the embedded audio and save it as a temporary file."""
        temp_file = "temp_alarm.mp3"
        with open(temp_file, "wb") as f:
            f.write(base64.b64decode(embedded_audio))
        return temp_file

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M")
            self.alarms.append(alarm_time)
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self):
        """Play the embedded alarm tone and handle stop/resound logic."""
        try:
            temp_file = self.save_embedded_audio()
            self.currently_playing = True
            self.stop_signal.clear()

            # Play the tone in a loop for up to 5 minutes
            start_time = time.time()
            while time.time() - start_time < 300:  # 5 minutes max
                if self.stop_signal.is_set():
                    break
                playsound(temp_file)

            self.currently_playing = False
            os.remove(temp_file)  # Clean up temporary file

            # Resound the alarm after 5 minutes if not stopped or snoozed
            if not self.stop_signal.is_set():
                self.auto_resound()

        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the alarm sound: {e}")

    def stop_alarm(self):
        """Stop the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()
            messagebox.showinfo("Stop", "Alarm stopped successfully!")
        

    def snooze_alarm(self):
        """Snooze the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()  # Stop the current alarm
            snoozed_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
            self.alarms.append(snoozed_time)
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm is currently sounding!")

    def auto_resound(self):
        """Automatically reschedule the alarm to sound after 5 minutes."""
        next_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
        self.alarms.append(next_time)
        self.alarm_listbox.insert(tk.END, f"Resounding alarm at {next_time}")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M")
            if current_time in self.alarms:
                self.alarms.remove(current_time)
                self.alarm_listbox.delete(0, tk.END)
                for alarm in self.alarms:
                    self.alarm_listbox.insert(tk.END, f"Alarm at {alarm}")
                threading.Thread(target=self.play_tone).start()
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.stop_signal.set()
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [34]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import datetime, timedelta
import threading
import time
import base64
from playsound import playsound
import os

# Base64 encoded strings for each MP3 file
embedded_audio_1 = "<Your base64 encoded string for alarm_1.mp3>"
embedded_audio_2 = "<Your base64 encoded string for alarm_2.mp3>"
embedded_audio_3 = "<Your base64 encoded string for morning_alarm.mp3>"
embedded_audio_4 = "<Your base64 encoded string for old_alarm_sound.mp3>"

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")

        # Alarm list
        self.alarms = []
        self.currently_playing = False  # Track if an alarm is currently sounding
        self.stop_signal = threading.Event()

        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Select Sound:").grid(row=1, column=0, padx=5, pady=10, columnspan=2)
        self.sound_options = ["Sound 1", "Sound 2", "Sound 3", "Sound 4"]
        self.selected_sound = tk.StringVar(root)
        self.selected_sound.set(self.sound_options[0])
        self.sound_menu = ttk.Combobox(root, textvariable=self.selected_sound, values=self.sound_options)
        self.sound_menu.grid(row=1, column=1, padx=10, pady=5, columnspan=2)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=0, pady=10)

        self.stop_button = tk.Button(root, text="Stop", command=self.stop_alarm)
        self.stop_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def save_embedded_audio(self, audio_data, filename):
        """Decode the embedded audio and save it as a temporary file."""
        with open(filename, "wb") as f:
            f.write(base64.b64decode(audio_data))
        return filename

    def get_audio_file(self):
        """Return the file path for the selected alarm sound."""
        if self.selected_sound.get() == "Sound 1":
            return self.save_embedded_audio(embedded_audio_1, "temp_alarm_1.mp3")
        elif self.selected_sound.get() == "Sound 2":
            return self.save_embedded_audio(embedded_audio_2, "temp_alarm_2.mp3")
        elif self.selected_sound.get() == "Sound 3":
            return self.save_embedded_audio(embedded_audio_3, "temp_alarm_3.mp3")
        else:
            return self.save_embedded_audio(embedded_audio_4, "temp_alarm_4.mp3")

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M")
            self.alarms.append(alarm_time)
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM.")

    def play_tone(self):
        """Play the embedded alarm tone and handle stop/resound logic."""
        try:
            audio_file = self.get_audio_file()
            self.currently_playing = True
            self.stop_signal.clear()

            # Play the tone in a loop for up to 5 minutes
            start_time = time.time()
            while time.time() - start_time < 300:  # 5 minutes max
                if self.stop_signal.is_set():
                    break
                playsound(audio_file)

            self.currently_playing = False
            os.remove(audio_file)  # Clean up temporary file

            # Resound the alarm after 5 minutes if not stopped or snoozed
            if not self.stop_signal.is_set():
                self.auto_resound()

        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the alarm sound: {e}")

    def stop_alarm(self):
        """Stop the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()
            messagebox.showinfo("Stop", "Alarm stopped successfully!")

    def snooze_alarm(self):
        """Snooze the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()  # Stop the current alarm
            snoozed_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
            self.alarms.append(snoozed_time)
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm is currently sounding!")

    def auto_resound(self):
        """Automatically reschedule the alarm to sound after 5 minutes."""
        next_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
        self.alarms.append(next_time)
        self.alarm_listbox.insert(tk.END, f"Resounding alarm at {next_time}")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M")
            if current_time in self.alarms:
                self.alarms.remove(current_time)
                self.alarm_listbox.delete(0, tk.END)
                for alarm in self.alarms:
                    self.alarm_listbox.insert(tk.END, f"Alarm at {alarm}")
                threading.Thread(target=self.play_tone).start()
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.stop_signal.set()
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [40]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import datetime, timedelta
import threading
import time
import base64
from playsound import playsound
import os

# Base64 encoded strings for each MP3 file (replace these with your actual encoded strings)
embedded_audio_1 = "<Base64 encoded string for alarm_1.mp3>"
embedded_audio_2 = "<Base64 encoded string for alarm_2.mp3>"
embedded_audio_3 = "<Base64 encoded string for morning_alarm.mp3>"
embedded_audio_4 = "<Base64 encoded string for old_alarm_sound.mp3>"

class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")

        # Alarm list
        self.alarms = []
        self.currently_playing = False  # Track if an alarm is currently sounding
        self.stop_signal = threading.Event()

        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Select Sound:").grid(row=1, column=0, padx=5, pady=10, columnspan=2)
        self.sound_options = ["Sound 1", "Sound 2", "Sound 3", "Sound 4"]
        self.selected_sound = tk.StringVar(root)
        self.selected_sound.set(self.sound_options[0])
        self.sound_menu = ttk.Combobox(root, textvariable=self.selected_sound, values=self.sound_options)
        self.sound_menu.grid(row=1, column=1, padx=10, pady=5, columnspan=2)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=0, pady=10)

        self.stop_button = tk.Button(root, text="Stop", command=self.stop_alarm)
        self.stop_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()


    

        def encode_audio_to_base64(file_path):
            with open(file_path, "rb") as audio_file:
                return base64.b64encode(audio_file.read()).decode('utf-8')

        # Encode each audio file
            encoded_audio_1 = encode_audio_to_base64("D:\\alarm_sound\\alarm_1.mp3")
            encoded_audio_2 = encode_audio_to_base64("D:\\alarm_sound\\alarm_2.mp3")
            encoded_audio_3 = encode_audio_to_base64("D:\\alarm_sound\\morning_alarm.mp3")
            encoded_audio_4 = encode_audio_to_base64("D:\\alarm_sound\\old_alarm_sound.mp3")

        # Print encoded audio strings (you can store these in your script)
            print(f"Embedded Audio 1:\n{encoded_audio_1}\n")
            print(f"Embedded Audio 2:\n{encoded_audio_2}\n")
            print(f"Embedded Audio 3:\n{encoded_audio_3}\n")
            print(f"Embedded Audio 4:\n{encoded_audio_4}\n")

    def encode_audio_to_base64(file_path):
        with open(file_path, "rb") as audio_file:
            return base64.b64encode(audio_file.read()).decode('utf-8')

        # Encode each audio file
        encoded_audio_1 = encode_audio_to_base64("D:\\alarm_sound\\alarm_1.mp3")
        encoded_audio_2 = encode_audio_to_base64("D:\\alarm_sound\\alarm_2.mp3")
        encoded_audio_3 = encode_audio_to_base64("D:\\alarm_sound\\morning_alarm.mp3")
        encoded_audio_4 = encode_audio_to_base64("D:\\alarm_sound\\old_alarm_sound.mp3")


    def save_embedded_audio(self, audio_data, filename):
        """Decode the embedded audio and save it as a temporary file."""
        with open(filename, "wb") as f:
            f.write(base64.b64decode(audio_data))
        return filename

    def get_audio_file(self):
        """Return the file path for the selected alarm sound."""
        if self.selected_sound.get() == "Sound 1":
            return self.save_embedded_audio(embedded_audio_1, "temp_alarm_1.mp3")
        elif self.selected_sound.get() == "Sound 2":
            return self.save_embedded_audio(embedded_audio_2, "temp_alarm_2.mp3")
        elif self.selected_sound.get() == "Sound 3":
            return self.save_embedded_audio(embedded_audio_3, "temp_alarm_3.mp3")
        else:
            return self.save_embedded_audio(embedded_audio_4, "temp_alarm_4.mp3")

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M")
            self.alarms.append(alarm_time)
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM.")

    def play_tone(self):
        """Play the embedded alarm tone and handle stop/resound logic."""
        try:
            audio_file = self.get_audio_file()
            self.currently_playing = True
            self.stop_signal.clear()

            # Play the tone in a loop for up to 5 minutes
            start_time = time.time()
            while time.time() - start_time < 300:  # 5 minutes max
                if self.stop_signal.is_set():
                    break
                playsound(audio_file)

            self.currently_playing = False
            os.remove(audio_file)  # Clean up temporary file

            # Resound the alarm after 5 minutes if not stopped or snoozed
            if not self.stop_signal.is_set():
                self.auto_resound()

        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the alarm sound: {e}")

    def stop_alarm(self):
        """Stop the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()
            messagebox.showinfo("Stop", "Alarm stopped successfully!")

    def snooze_alarm(self):
        """Snooze the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()  # Stop the current alarm
            snoozed_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
            self.alarms.append(snoozed_time)
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm is currently sounding!")

    def auto_resound(self):
        """Automatically reschedule the alarm to sound after 5 minutes."""
        next_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
        self.alarms.append(next_time)
        self.alarm_listbox.insert(tk.END, f"Resounding alarm at {next_time}")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M")
            if current_time in self.alarms:
                self.alarms.remove(current_time)
                self.alarm_listbox.delete(0, tk.END)
                for alarm in self.alarms:
                    self.alarm_listbox.insert(tk.END, f"Alarm at {alarm}")
                threading.Thread(target=self.play_tone).start()
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.stop_signal.set()
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [44]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import datetime, timedelta
import threading
import time
from playsound import playsound
import os


class AlarmApp:
    def __init__(selfa, root):
        self.root = root
        self.root.title("Alarm Clock")

        # Alarm list
        self.alarms = []
        self.currently_playing = False  # Track if an alarm is currently sounding
        self.stop_signal = threading.Event()

        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Select Sound:").grid(row=1, column=0, padx=5, pady=10, columnspan=2)
        self.sound_options = ["Sound 1", "Sound 2", "Sound 3", "Sound 4"]
        self.selected_sound = tk.StringVar(root)
        self.selected_sound.set(self.sound_options[0])
        self.sound_menu = ttk.Combobox(root, textvariable=self.selected_sound, values=self.sound_options)
        self.sound_menu.grid(row=1, column=1, padx=10, pady=5, columnspan=2)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=0, pady=10)

        self.stop_button = tk.Button(root, text="Stop", command=self.stop_alarm)
        self.stop_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    
    def add_alarm(self):
        alarm_time = self.time_entry.get()
        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M")
            self.alarms.append(alarm_time)
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM.")

    def play_tone(self):
        """Play the embedded alarm tone and handle stop/resound logic."""
        try:
            audio_file = self.get_audio_file()
            self.currently_playing = True
            self.stop_signal.clear()

            # Play the tone in a loop for up to 5 minutes
            start_time = time.time()
            while time.time() - start_time < 300:  # 5 minutes max
                if self.stop_signal.is_set():
                    break
                playsound(audio_file)

            self.currently_playing = False
            os.remove(audio_file)  # Clean up temporary file

            # Resound the alarm after 5 minutes if not stopped or snoozed
            if not self.stop_signal.is_set():
                self.auto_resound()

        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the alarm sound: {e}")

    def stop_alarm(self):
        """Stop the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()
            messagebox.showinfo("Stop", "Alarm stopped successfully!")

    def snooze_alarm(self):
        """Snooze the currently sounding alarm."""
        if self.currently_playing:
            self.stop_signal.set()  # Stop the current alarm
            snoozed_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
            self.alarms.append(snoozed_time)
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm is currently sounding!")

    def auto_resound(self):
        """Automatically reschedule the alarm to sound after 5 minutes."""
        next_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M")
        self.alarms.append(next_time)
        self.alarm_listbox.insert(tk.END, f"Resounding alarm at {next_time}")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M")
            if current_time in self.alarms:
                self.alarms.remove(current_time)
                self.alarm_listbox.delete(0, tk.END)
                for alarm in self.alarms:
                    self.alarm_listbox.insert(tk.END, f"Alarm at {alarm}")
                threading.Thread(target=self.play_tone).start()
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.stop_signal.set()
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [50]:
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime, timedelta
import threading
import time  
!pip install playsound
 
# Then run your script again as usual.
from playsound import playsound





class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root,width=20)
        root.columnconfigure(1, weight=0)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Tone").grid(row=1, column=0, padx=10, pady=5)
        self.tone_path = tk.StringVar()
        self.tone_entry = tk.Entry(root, textvariable=self.tone_path)
        self.tone_entry.grid(row=1, column=1, padx=10, pady=5)
        
        self.browse_button = tk.Button(root, text="Browse", command=self.browse_tone)
        self.browse_button.grid(row=1, column=2, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=3, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def browse_tone(self):
        filepath = filedialog.askopenfilename(filetypes=[("Audio Files", "*.mp3;*.wav")])
        if filepath:
            self.tone_path.set(filepath)

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.tone_path.get()

        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            if tone:
                self.alarms.append((alarm_time, tone))
                self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with tone {tone}")
                messagebox.showinfo("Success", "Alarm added successfully!")
            else:
                messagebox.showwarning("Error", "Please select a tone!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self, tone):
        try:
            playsound(tone)
        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the selected tone: {e}")

    def snooze_alarm(self):
        selected = self.alarm_listbox.curselection()
        if selected:
            alarm = self.alarms[selected[0]]
            alarm_time = datetime.strptime(alarm[0], "%H:%M:%S") + timedelta(minutes=5)
            snoozed_time = alarm_time.time().strftime("%H:%M:%S")

            self.alarms.append((snoozed_time, alarm[1]))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time} with tone {alarm[1]}")

            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm selected to snooze!")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm in self.alarms:
                if alarm[0] == current_time:
                    self.play_tone(alarm[1])
                    messagebox.showinfo("Alarm", f"Alarm ringing at {alarm[0]}!")
                    self.alarms.remove(alarm)
                    self.alarm_listbox.delete(0, tk.END)
                    for a in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {a[0]} with tone {a[1]}")
            time.sleep(1)

    def stop_alarm(self):

    def on_close(self):
        self.running = False
        self.check_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()

In [52]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import datetime, timedelta
import threading
import time
from playsound import playsound
import os


class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        self.currently_playing = False
        self.stop_signal = threading.Event()
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Select Tone:").grid(row=1, column=0, padx=10, pady=5)
        self.sound_options = os.listdir("sounds/")  # List all files in the sounds directory
        self.selected_sound = tk.StringVar()
        self.selected_sound.set(self.sound_options[0] if self.sound_options else "No Sounds")
        self.sound_menu = ttk.Combobox(root, textvariable=self.selected_sound, values=self.sound_options)
        self.sound_menu.grid(row=1, column=1, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=0, pady=10)

        self.stop_button = tk.Button(root, text="Stop", command=self.stop_alarm)
        self.stop_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.selected_sound.get()

        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            self.alarms.append((alarm_time, tone))
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with tone {tone}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self, tone):
        """Play the selected alarm tone for up to 5 minutes."""
        try:
            self.currently_playing = True
            self.stop_signal.clear()
            start_time = time.time()

            while time.time() - start_time < 300:  # Play for 5 minutes max
                if self.stop_signal.is_set():
                    break
                playsound(f"sounds/{tone}")

            self.currently_playing = False

            if not self.stop_signal.is_set():
                self.auto_resound()

        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the selected tone: {e}")

    def stop_alarm(self):
        """Stop the currently ringing alarm."""
        if self.currently_playing:
            self.stop_signal.set()
            messagebox.showinfo("Stop", "Alarm stopped successfully!")

    def snooze_alarm(self):
        """Snooze the currently ringing alarm."""
        if self.currently_playing:
            self.stop_signal.set()  # Stop the current alarm
            snoozed_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M:%S")
            self.alarms.append((snoozed_time, self.selected_sound.get()))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm is currently sounding!")

    def auto_resound(self):
        """Automatically reschedule the alarm to sound after 5 minutes."""
        next_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M:%S")
        self.alarms.append((next_time, self.selected_sound.get()))
        self.alarm_listbox.insert(tk.END, f"Resounding alarm at {next_time}")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm in self.alarms:
                if alarm[0] == current_time:
                    self.alarms.remove(alarm)
                    self.alarm_listbox.delete(0, tk.END)
                    for a in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {a[0]} with tone {a[1]}")
                    threading.Thread(target=self.play_tone, args=(alarm[1],)).start()
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.stop_signal.set()
        self.check_thread.join()
        self.root.destroy()


if __name__ == "__main__":
    # Ensure a sounds directory exists with audio files
    if not os.path.exists("sounds/"):
        os.makedirs("sounds/")

    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


In [73]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import datetime, timedelta
import threading
import time
from playsound import playsound
import os


class AlarmApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alarm Clock")
        
        # Alarm list
        self.alarms = []
        self.currently_playing = False
        self.stop_signal = threading.Event()
        
        # GUI Components
        tk.Label(root, text="Set Alarm (HH:MM:SS):").grid(row=0, column=0, padx=5, pady=10)
        self.time_entry = tk.Entry(root, width=20)
        self.time_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(root, text="Select Tone:").grid(row=1, column=0, padx=10, pady=5)
       
        self.sound_options = os.listdir("alarm_sound/")  # List all files in the sounds directory
        print("Audio files in 'sounds' directory:")
        for file in sound_files:
            print(file)
        self.selected_sound = tk.StringVar()
        self.selected_sound.set(self.sound_options[0] if self.sound_options else "No Sounds")
        self.sound_menu = ttk.Combobox(root, textvariable=self.selected_sound, values=self.sound_options)
        self.sound_menu.grid(row=1, column=1, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add Alarm", command=self.add_alarm)
        self.add_button.grid(row=2, column=1, pady=10)

        self.alarm_listbox = tk.Listbox(root, width=50)
        self.alarm_listbox.grid(row=3, column=0, columnspan=2, pady=10)

        self.snooze_button = tk.Button(root, text="Snooze", command=self.snooze_alarm)
        self.snooze_button.grid(row=4, column=0, pady=10)

        self.stop_button = tk.Button(root, text="Stop", command=self.stop_alarm)
        self.stop_button.grid(row=4, column=1, pady=10)

        # Start the alarm checking thread
        self.running = True
        self.check_thread = threading.Thread(target=self.check_alarms)
        self.check_thread.daemon = True
        self.check_thread.start()

    def add_alarm(self):
        alarm_time = self.time_entry.get()
        tone = self.selected_sound.get()

        try:
            # Validate time format
            datetime.strptime(alarm_time, "%H:%M:%S")
            self.alarms.append((alarm_time, tone))
            self.alarm_listbox.insert(tk.END, f"Alarm at {alarm_time} with tone {tone}")
            messagebox.showinfo("Success", "Alarm added successfully!")
        except ValueError:
            messagebox.showwarning("Error", "Invalid time format! Use HH:MM:SS.")

    def play_tone(self, tone):
        """Play the selected alarm tone for up to 5 minutes."""
        try:
            self.currently_playing = True
            self.stop_signal.clear()
            start_time = time.time()

            while time.time() - start_time < 300:  # Play for 5 minutes max
                if self.stop_signal.is_set():
                    break
                playsound(f"sounds/{tone}")

            self.currently_playing = False

            if not self.stop_signal.is_set():
                self.auto_resound()

        except Exception as e:
            messagebox.showwarning("Error", f"Unable to play the selected tone: {e}")

    def stop_alarm(self):
        """Stop the currently ringing alarm."""
        if self.currently_playing:
            self.stop_signal.set()
            messagebox.showinfo("Stop", "Alarm stopped successfully!")

    def snooze_alarm(self):
        """Snooze the currently ringing alarm."""
        if self.currently_playing:
            self.stop_signal.set()  # Stop the current alarm
            snoozed_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M:%S")
            self.alarms.append((snoozed_time, self.selected_sound.get()))
            self.alarm_listbox.insert(tk.END, f"Snoozed alarm at {snoozed_time}")
            messagebox.showinfo("Snooze", f"Alarm snoozed to {snoozed_time}")
        else:
            messagebox.showwarning("Error", "No alarm is currently sounding!")

    def auto_resound(self):
        """Automatically reschedule the alarm to sound after 5 minutes."""
        next_time = (datetime.now() + timedelta(minutes=5)).strftime("%H:%M:%S")
        self.alarms.append((next_time, self.selected_sound.get()))
        self.alarm_listbox.insert(tk.END, f"Resounding alarm at {next_time}")

    def check_alarms(self):
        while self.running:
            current_time = datetime.now().strftime("%H:%M:%S")
            for alarm in self.alarms:
                if alarm[0] == current_time:
                    self.alarms.remove(alarm)
                    self.alarm_listbox.delete(0, tk.END)
                    for a in self.alarms:
                        self.alarm_listbox.insert(tk.END, f"Alarm at {a[0]} with tone {a[1]}")
                    threading.Thread(target=self.play_tone, args=(alarm[1],)).start()
            time.sleep(1)

    def on_close(self):
        self.running = False
        self.stop_signal.set()
        self.check_thread.join()
        self.root.destroy()


if __name__ == "__main__":
    # Ensure a sounds directory exists with audio files
    if not os.path.exists("alarm_sound/"):
        os.makedirs("alarm_sound/")
        import os



print("Audio files in 'sounds' directory:")
for file in sound_files:
    print(file)


    root = tk.Tk()
    app = AlarmApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)
    root.mainloop()


Audio files in 'sounds' directory:


NameError: name 'sound_files' is not defined

In [75]:
print("Audio files in 'sounds' directory:")
for file in sound_files:
    print(file)

Audio files in 'sounds' directory:


NameError: name 'sound_files' is not defined